In [1]:
import os
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
import pysam
from collections import Counter
from matplotlib import cm
from matplotlib.colors import ListedColormap

import io
from pysam import VariantFile

from sklearn.metrics import precision_score, recall_score, average_precision_score, precision_recall_curve

In [2]:
tab20 = cm.get_cmap('tab10', 8)
newcmap_list = []
for i in range(4):
    newcmap_list.append(tab20.colors[i])
for i in range(4):
    newcmap_list.append(np.array(list(tab20.colors[i][:-1]) +[0.3]))

newcmap = ListedColormap(newcmap_list, name='newcmap')
color_list = ['tab:blue', 'tab:orange', 'tab:green', 'tab:red', 'tab:purple']

In [3]:
def read_vcf(path):
    with open(path, 'r') as f:
        lines = [l for l in f if not l.startswith('##')]
    res = pd.read_csv(
        io.StringIO(''.join(lines)),
        dtype={'#CHROM': str, 'POS': int, 'ID': str, 'REF': str, 'ALT': str,
               'QUAL': str, 'FILTER': str, 'INFO': str},
        sep='\t'
    ).rename(columns={'#CHROM': 'CHROM'})
    return res

foo = lambda x: pd.Series(x.split('CALLERS=')[1].split(';')[0])
foo2 = lambda x: pd.Series(x.split('TYPE=')[1].split(';')[0] if len(x.split('TYPE=')) > 1 else np.nan)

In [4]:
methods = ['freebayes', 'mutect2', 'strelka2', 'vardict', 'varscan']

# Select plasma sample

In [5]:
sample = '809'
#sample = '986'

In [6]:
if sample == '809':
    plasmasample1 = '809_110914'
    plasmasample2 = '809_030915'
    tumorsample1 = '809_290714-T1W'
elif sample == '986':
    plasmasample1 = '986_100215'
    plasmasample2 = '986_261016'
    tumorsample1 = '986_100215-T1W'

# Load SNV calls for plasma sample and matching mixed samples

In [ ]:
vcf_pd_0 = read_vcf("../data/2015-07-31_CRC-"+plasmasample1+"-1-0/CRC-"+plasmasample1+"-1-0-ensemble-annotated.vcf")
vcf_pd_0.head()

In [ ]:
vcf_pd_0 = read_vcf("../data/2015-07-31_CRC-"+plasmasample1+"-1-0/CRC-"+plasmasample1+"-1-0-ensemble-annotated.vcf")
vcf_pd_1 = read_vcf("../data/2015-07-31_CRC-"+plasmasample1+"-1-05775/CRC-"+plasmasample1+"-1-05775-ensemble-annotated.vcf")
vcf_pd_2 = read_vcf("../data/2015-07-31_CRC-"+plasmasample1+"-075-06738/CRC-"+plasmasample1+"-075-06738-ensemble-annotated.vcf")
vcf_pd_3 = read_vcf("../data/2015-07-31_CRC-"+plasmasample1+"-05-07701/CRC-"+plasmasample1+"-05-07701-ensemble-annotated.vcf")
vcf_pd_4 = read_vcf("../data/2015-07-31_CRC-"+plasmasample1+"-025-08663/CRC-"+plasmasample1+"-025-08663-ensemble-annotated.vcf")

vcf_pd_0['callers'] = vcf_pd_0['INFO'].apply(foo)
vcf_pd_0['type'] = np.nan
vcf_pd_0['type'][vcf_pd_0['ALT'].str.len() - vcf_pd_0['REF'].str.len() == 0] = 'SNV'
vcf_pd_0['type'][vcf_pd_0['ALT'].str.len() - vcf_pd_0['REF'].str.len() > 0] = 'INS'
vcf_pd_0['type'][vcf_pd_0['ALT'].str.len() - vcf_pd_0['REF'].str.len() < 0] = 'DEL'
vcf_pd_0['type'][vcf_pd_0['ID'].str.contains('rs')] = 'SNP'
vcf_pd_0['freebayes'] = vcf_pd_0['INFO'].str.contains('freebayes')
vcf_pd_0['vardict'] = vcf_pd_0['INFO'].str.contains('vardict')
vcf_pd_0['varscan'] = vcf_pd_0['INFO'].str.contains('varscan')
vcf_pd_0['mutect2'] = vcf_pd_0['INFO'].str.contains('mutect2')
vcf_pd_0['strelka2'] = vcf_pd_0['INFO'].str.contains('strelka2')

vcf_pd_1['callers'] = vcf_pd_1['INFO'].apply(foo)
vcf_pd_1['type'] = np.nan
vcf_pd_1['type'][vcf_pd_1['ALT'].str.len() - vcf_pd_1['REF'].str.len() == 0] = 'SNV'
vcf_pd_1['type'][vcf_pd_1['ALT'].str.len() - vcf_pd_1['REF'].str.len() > 0] = 'INS'
vcf_pd_1['type'][vcf_pd_1['ALT'].str.len() - vcf_pd_1['REF'].str.len() < 0] = 'DEL'
vcf_pd_1['type'][vcf_pd_1['ID'].str.contains('rs')] = 'SNP'
vcf_pd_1['freebayes'] = vcf_pd_1['INFO'].str.contains('freebayes')
vcf_pd_1['vardict'] = vcf_pd_1['INFO'].str.contains('vardict')
vcf_pd_1['varscan'] = vcf_pd_1['INFO'].str.contains('varscan')
vcf_pd_1['mutect2'] = vcf_pd_1['INFO'].str.contains('mutect2')
vcf_pd_1['strelka2'] = vcf_pd_1['INFO'].str.contains('strelka2')

vcf_pd_2['callers'] = vcf_pd_2['INFO'].apply(foo)
vcf_pd_2['type'] = np.nan
vcf_pd_2['type'][vcf_pd_2['ALT'].str.len() - vcf_pd_2['REF'].str.len() == 0] = 'SNV'
vcf_pd_2['type'][vcf_pd_2['ALT'].str.len() - vcf_pd_2['REF'].str.len() > 0] = 'INS'
vcf_pd_2['type'][vcf_pd_2['ALT'].str.len() - vcf_pd_2['REF'].str.len() < 0] = 'DEL'
vcf_pd_2['type'][vcf_pd_2['ID'].str.contains('rs')] = 'SNP'
vcf_pd_2['freebayes'] = vcf_pd_2['INFO'].str.contains('freebayes')
vcf_pd_2['vardict'] = vcf_pd_2['INFO'].str.contains('vardict')
vcf_pd_2['varscan'] = vcf_pd_2['INFO'].str.contains('varscan')
vcf_pd_2['mutect2'] = vcf_pd_2['INFO'].str.contains('mutect2')
vcf_pd_2['strelka2'] = vcf_pd_2['INFO'].str.contains('strelka2')

vcf_pd_3['callers'] = vcf_pd_3['INFO'].apply(foo)
vcf_pd_3['type'] = np.nan
vcf_pd_3['type'][vcf_pd_3['ALT'].str.len() - vcf_pd_3['REF'].str.len() == 0] = 'SNV'
vcf_pd_3['type'][vcf_pd_3['ALT'].str.len() - vcf_pd_3['REF'].str.len() > 0] = 'INS'
vcf_pd_3['type'][vcf_pd_3['ALT'].str.len() - vcf_pd_3['REF'].str.len() < 0] = 'DEL'
vcf_pd_3['type'][vcf_pd_3['ID'].str.contains('rs')] = 'SNP'
vcf_pd_3['freebayes'] = vcf_pd_3['INFO'].str.contains('freebayes')
vcf_pd_3['vardict'] = vcf_pd_3['INFO'].str.contains('vardict')
vcf_pd_3['varscan'] = vcf_pd_3['INFO'].str.contains('varscan')
vcf_pd_3['mutect2'] = vcf_pd_3['INFO'].str.contains('mutect2')
vcf_pd_3['strelka2'] = vcf_pd_3['INFO'].str.contains('strelka2')

vcf_pd_4['callers'] = vcf_pd_4['INFO'].apply(foo)
vcf_pd_4['type'] = np.nan
vcf_pd_4['type'][vcf_pd_4['ALT'].str.len() - vcf_pd_4['REF'].str.len() == 0] = 'SNV'
vcf_pd_4['type'][vcf_pd_4['ALT'].str.len() - vcf_pd_4['REF'].str.len() > 0] = 'INS'
vcf_pd_4['type'][vcf_pd_4['ALT'].str.len() - vcf_pd_4['REF'].str.len() < 0] = 'DEL'
vcf_pd_4['type'][vcf_pd_4['ID'].str.contains('rs')] = 'SNP'
vcf_pd_4['freebayes'] = vcf_pd_4['INFO'].str.contains('freebayes')
vcf_pd_4['vardict'] = vcf_pd_4['INFO'].str.contains('vardict')
vcf_pd_4['varscan'] = vcf_pd_4['INFO'].str.contains('varscan')
vcf_pd_4['mutect2'] = vcf_pd_4['INFO'].str.contains('mutect2')
vcf_pd_4['strelka2'] = vcf_pd_4['INFO'].str.contains('strelka2')

sample_0 = vcf_pd_0[['CHROM', 'POS', 'REF', 'ALT', 'QUAL', 'FILTER', 'type', 'freebayes', 'vardict', 'varscan', 'mutect2', 'strelka2']]
sample_0['CHROM_POS'] = sample_0['CHROM'].astype('str').str.cat(sample_0['POS'].astype('str'),sep="_")
sample_0.set_index('CHROM_POS', inplace = True)

sample_1 = vcf_pd_1[['CHROM', 'POS', 'REF', 'ALT', 'QUAL', 'type', 'freebayes', 'vardict', 'varscan', 'mutect2', 'strelka2']]
sample_1['CHROM_POS'] = sample_1['CHROM'].astype('str').str.cat(sample_1['POS'].astype('str'),sep="_")
sample_1.set_index('CHROM_POS', inplace = True)

sample_2 = vcf_pd_2[['CHROM', 'POS', 'REF', 'ALT', 'QUAL', 'type', 'freebayes', 'vardict', 'varscan', 'mutect2', 'strelka2']]
sample_2['CHROM_POS'] = sample_2['CHROM'].astype('str').str.cat(sample_2['POS'].astype('str'),sep="_")
sample_2.set_index('CHROM_POS', inplace = True)

sample_3 = vcf_pd_3[['CHROM', 'POS', 'REF', 'ALT', 'QUAL', 'type', 'freebayes', 'vardict', 'varscan', 'mutect2', 'strelka2']]
sample_3['CHROM_POS'] = sample_3['CHROM'].astype('str').str.cat(sample_3['POS'].astype('str'),sep="_")
sample_3.set_index('CHROM_POS', inplace = True)

sample_4 = vcf_pd_4[['CHROM', 'POS', 'REF', 'ALT', 'QUAL', 'type', 'freebayes', 'vardict', 'varscan', 'mutect2', 'strelka2']]
sample_4['CHROM_POS'] = sample_4['CHROM'].astype('str').str.cat(sample_4['POS'].astype('str'),sep="_")
sample_4.set_index('CHROM_POS', inplace = True)

sample_4.head()

In [ ]:
vcf_pd_0_bis = read_vcf("../data/2015-07-31_CRC-"+plasmasample2+"-1-0/CRC-"+plasmasample2+"-1-0-ensemble-annotated.vcf")

vcf_pd_0_bis['type'] = np.nan
vcf_pd_0_bis['type'][vcf_pd_0_bis['ALT'].str.len() - vcf_pd_0_bis['REF'].str.len() == 0] = 'SNV'
vcf_pd_0_bis['type'][vcf_pd_0_bis['ALT'].str.len() - vcf_pd_0_bis['REF'].str.len() > 0] = 'INS'
vcf_pd_0_bis['type'][vcf_pd_0_bis['ALT'].str.len() - vcf_pd_0_bis['REF'].str.len() < 0] = 'DEL'
vcf_pd_0_bis['type'][vcf_pd_0_bis['ID'].str.contains('rs')] = 'SNP'
vcf_pd_0_bis['freebayes'] = vcf_pd_0_bis['INFO'].str.contains('freebayes')
vcf_pd_0_bis['vardict'] = vcf_pd_0_bis['INFO'].str.contains('vardict')
vcf_pd_0_bis['varscan'] = vcf_pd_0_bis['INFO'].str.contains('varscan')
vcf_pd_0_bis['mutect2'] = vcf_pd_0_bis['INFO'].str.contains('mutect2')
vcf_pd_0_bis['strelka2'] = vcf_pd_0_bis['INFO'].str.contains('strelka2')

sample_0_bis = vcf_pd_0_bis[['CHROM', 'POS', 'REF', 'ALT', 'QUAL', 'type', 'freebayes', 'vardict', 'varscan', 'mutect2', 'strelka2']]
sample_0_bis['CHROM_POS'] = sample_0_bis['CHROM'].astype('str').str.cat(sample_0_bis['POS'].astype('str'),sep="_")
sample_0_bis.set_index('CHROM_POS', inplace = True)
sample_0_bis.head()

sample0 = sample_0.copy()
sample0['date'] = '110914'
sample0bis = sample_0_bis.copy()
sample0bis['date'] = '030915'
sample_ref = pd.concat([sample0, sample0bis], join='inner')
l1 = list(sample_ref[sample_ref['date'] == '110914'].index)
l2 = list(sample_ref[sample_ref['date'] == '030915'].index)
print(len(l1), len(l2))
print(len(list(set(set(l1) ^ set(l2)))))
print(len(list(set(set(l1) & set(l2)))))

In [ ]:
vcf_pd_h = read_vcf("../data/2015-07-31_Merged_Healthy/Merged_Healthy-ensemble-annotated.vcf")

vcf_pd_h['type'] = np.nan
vcf_pd_h['type'][vcf_pd_h['ALT'].str.len() - vcf_pd_h['REF'].str.len() == 0] = 'SNV'
vcf_pd_h['type'][vcf_pd_h['ALT'].str.len() - vcf_pd_h['REF'].str.len() > 0] = 'INS'
vcf_pd_h['type'][vcf_pd_h['ALT'].str.len() - vcf_pd_h['REF'].str.len() < 0] = 'DEL'
vcf_pd_h['type'][vcf_pd_h['ID'].str.contains('rs')] = 'SNP'
vcf_pd_h['freebayes'] = vcf_pd_h['INFO'].str.contains('freebayes')
vcf_pd_h['vardict'] = vcf_pd_h['INFO'].str.contains('vardict')
vcf_pd_h['varscan'] = vcf_pd_h['INFO'].str.contains('varscan')
vcf_pd_h['mutect2'] = vcf_pd_h['INFO'].str.contains('mutect2')
vcf_pd_h['strelka2'] = vcf_pd_h['INFO'].str.contains('strelka2')

sample_h = vcf_pd_h[['CHROM', 'POS', 'REF', 'ALT', 'QUAL', 'type', 'freebayes', 'vardict', 'varscan', 'mutect2', 'strelka2']]
sample_h['CHROM_POS'] = sample_h['CHROM'].astype('str').str.cat(sample_h['POS'].astype('str'),sep="_")
sample_h.set_index('CHROM_POS', inplace = True)
sample_h = sample_h[sample_h['CHROM'] == '22']
sample_h.head()

In [ ]:
vcf_pd_t = read_vcf("../data/2015-07-31_NCC_CRC-"+tumorsample1+"/NCC_CRC-"+tumorsample1+"-ensemble-annotated.vcf")

vcf_pd_t['type'] = np.nan
vcf_pd_t['type'][vcf_pd_t['ALT'].str.len() - vcf_pd_t['REF'].str.len() == 0] = 'SNV'
vcf_pd_t['type'][vcf_pd_t['ALT'].str.len() - vcf_pd_t['REF'].str.len() > 0] = 'INS'
vcf_pd_t['type'][vcf_pd_t['ALT'].str.len() - vcf_pd_t['REF'].str.len() < 0] = 'DEL'
vcf_pd_t['type'][vcf_pd_t['ID'].str.contains('rs')] = 'SNP'
vcf_pd_t['freebayes'] = vcf_pd_t['INFO'].str.contains('freebayes')
vcf_pd_t['vardict'] = vcf_pd_t['INFO'].str.contains('vardict')
vcf_pd_t['varscan'] = vcf_pd_t['INFO'].str.contains('varscan')
vcf_pd_t['mutect2'] = vcf_pd_t['INFO'].str.contains('mutect2')
vcf_pd_t['strelka2'] = vcf_pd_t['INFO'].str.contains('strelka2')

sample_t = vcf_pd_t[['CHROM', 'POS', 'REF', 'ALT', 'QUAL', 'type', 'freebayes', 'vardict', 'varscan', 'mutect2', 'strelka2']]
sample_t['CHROM_POS'] = sample_t['CHROM'].astype('str').str.cat(sample_t['POS'].astype('str'),sep="_")
sample_t.set_index('CHROM_POS', inplace = True)
sample_t = sample_t[sample_t['CHROM'] == '22']
print(sample_t.shape)
sample_t = sample_t.loc[list(sample_t.index[~sample_t.index.isin(sample_h.index)])]
print(sample_t.shape)

l1 = list(sample_t.index)
l2 = list(sample_0.index)
print(len(l1), len(l2))
print(len(list(set(set(l1) & set(l2)))))

sample_t.head()

# Estimated tumor burden of mixed samples

In [ ]:
samples_tf = {
    'sample_0': float(list(pd.read_csv("../data/2015-07-31_CRC-"+plasmasample1+"-1-0/estimated_tf.txt").columns)[0]),
    'sample_1': float(list(pd.read_csv("../data/2015-07-31_CRC-"+plasmasample1+"-1-05775/estimated_tf.txt").columns)[0]),
    'sample_2': float(list(pd.read_csv("../data/2015-07-31_CRC-"+plasmasample1+"-075-06738/estimated_tf.txt").columns)[0]),
    'sample_3': float(list(pd.read_csv("../data/2015-07-31_CRC-"+plasmasample1+"-05-07701/estimated_tf.txt").columns)[0]),
    'sample_4': float(list(pd.read_csv("../data/2015-07-31_CRC-"+plasmasample1+"-025-08663/estimated_tf.txt").columns)[0]),
}

print(samples_tf)

# Number of detections detected

In [ ]:
numbersnvs_pd = pd.DataFrame()

for si, s in enumerate([sample_0, sample_1, sample_2, sample_3, sample_4]):
    nb_snv = []
    for method in methods:
        #if si == 4:
        #    si = 't'
       # print('sample '+ str(si) + ': ', method, s[s[method] == True].shape[0])
        nb_snv.append(s[s[method] == True].shape[0])
    if si == 0:
        numbersnvs_pd = pd.DataFrame.from_dict({'sample_'+ str(si): nb_snv}).T
        numbersnvs_pd.columns = methods
    else:
        numbersnvs_pd.loc['sample_'+ str(si)] = nb_snv
    numbersnvs_pd = numbersnvs_pd.rename(index=samples_tf)

numbersnvs_pd.plot(style='.-', logx=True, xlim=(10e-1, 10e-3))
#sns.catplot(x='tumor burden', y='coverage', kind='point', data=coverage_pd,
#            order=sorted(coverage_pd['tumor burden'].unique(), reverse=True))
        
numbersnvs_pd

In [ ]:
numbersnvs_pd = pd.DataFrame()

for si, s in enumerate([sample_0, sample_1, sample_2, sample_3, sample_4]):
    nb_snv = []
    for method in methods:
        #if si == 4:
        #    si = 't'
       # print('sample '+ str(si) + ': ', method, s[s[method] == True].shape[0])
        nb_snv.append(s[(s[method] == True) & ((s['type'] == 'INS') | (s['type'] == 'DEL'))].shape[0])
    if si == 0:
        numbersnvs_pd = pd.DataFrame.from_dict({'sample_'+ str(si): nb_snv}).T
        numbersnvs_pd.columns = methods
    else:
        numbersnvs_pd.loc['sample_'+ str(si)] = nb_snv
    numbersnvs_pd = numbersnvs_pd.rename(index=samples_tf)

numbersnvs_pd.plot(style='.-', logx=True, xlim=(10e-1, 10e-3))
#sns.catplot(x='tumor burden', y='coverage', kind='point', data=coverage_pd,
#            order=sorted(coverage_pd['tumor burden'].unique(), reverse=True))
        
numbersnvs_pd

In [ ]:
numbersnvs_pd = pd.DataFrame()

for si, s in enumerate([sample_0, sample_1, sample_2, sample_3, sample_4]):
    nb_snv = []
    for method in methods:
        #if si == 4:
        #    si = 't'
       # print('sample '+ str(si) + ': ', method, s[s[method] == True].shape[0])
        nb_snv.append(s[(s[method] == True) & (s['type'] == 'SNV')].shape[0])
    if si == 0:
        numbersnvs_pd = pd.DataFrame.from_dict({'sample_'+ str(si): nb_snv}).T
        numbersnvs_pd.columns = methods
    else:
        numbersnvs_pd.loc['sample_'+ str(si)] = nb_snv
    numbersnvs_pd = numbersnvs_pd.rename(index=samples_tf)

numbersnvs_pd.plot(style='.-', logx=True, xlim=(10e-1, 10e-3))
#sns.catplot(x='tumor burden', y='coverage', kind='point', data=coverage_pd,
#            order=sorted(coverage_pd['tumor burden'].unique(), reverse=True))
        
numbersnvs_pd

In [ ]:
numbersnvs_pd = pd.DataFrame()

for si, s in enumerate([sample_0, sample_1, sample_2, sample_3, sample_4]):
    nb_snv = []
    for method in methods:
        #if si == 4:
        #    si = 't'
       # print('sample '+ str(si) + ': ', method, s[s[method] == True].shape[0])
        nb_snv.append(s[(s[method] == True) & (s['type'] == 'SNP')].shape[0])
    if si == 0:
        numbersnvs_pd = pd.DataFrame.from_dict({'sample_'+ str(si): nb_snv}).T
        numbersnvs_pd.columns = methods
    else:
        numbersnvs_pd.loc['sample_'+ str(si)] = nb_snv
    numbersnvs_pd = numbersnvs_pd.rename(index=samples_tf)

numbersnvs_pd.plot(style='.-', logx=True, xlim=(10e-1, 10e-3))
#sns.catplot(x='tumor burden', y='coverage', kind='point', data=coverage_pd,
#            order=sorted(coverage_pd['tumor burden'].unique(), reverse=True))
        
numbersnvs_pd

# Coverage variability

In [ ]:
samples_cov_tf = {
    'sample_0': [float(list(pd.read_csv("../data/2015-07-31_CRC-"+plasmasample1+"-1-0/coverage.txt").columns)[0]),
                np.round(float(list(pd.read_csv("../data/2015-07-31_CRC-"+plasmasample1+"-1-0/estimated_tf.txt").columns)[0]),2)],
    'sample_1': [float(list(pd.read_csv("../data/2015-07-31_CRC-"+plasmasample1+"-1-05775/coverage.txt").columns)[0]),
                np.round(float(list(pd.read_csv("../data/2015-07-31_CRC-"+plasmasample1+"-1-05775/estimated_tf.txt").columns)[0]),2)],
    'sample_2': [float(list(pd.read_csv("../data/2015-07-31_CRC-"+plasmasample1+"-075-06738/coverage.txt").columns)[0]),
                np.round(float(list(pd.read_csv("../data/2015-07-31_CRC-"+plasmasample1+"-075-06738/estimated_tf.txt").columns)[0]),2)],
    'sample_3': [float(list(pd.read_csv("../data/2015-07-31_CRC-"+plasmasample1+"-05-07701/coverage.txt").columns)[0]),
                np.round(float(list(pd.read_csv("../data/2015-07-31_CRC-"+plasmasample1+"-05-07701/estimated_tf.txt").columns)[0]),2)],
    'sample_4': [float(list(pd.read_csv("../data/2015-07-31_CRC-"+plasmasample1+"-025-08663/coverage.txt").columns)[0]),
                np.round(float(list(pd.read_csv("../data/2015-07-31_CRC-"+plasmasample1+"-025-08663/estimated_tf.txt").columns)[0]),2)],
    
}

print(samples_cov_tf)

coverage_pd = pd.DataFrame.from_dict(samples_cov_tf).T
coverage_pd.columns = ['coverage', 'tumor burden']
sns.catplot(x='tumor burden', y='coverage', data=coverage_pd,
            order=sorted(coverage_pd['tumor burden'].unique(), reverse=True), color='magenta')

        
coverage_pd

# Load SMURF calls

In [ ]:
smurf_snv = pd.read_csv('../data/SMURF/NCC_CRC-'+plasmasample1+'-CW/snv-predicted.txt', sep='\t')
smurf_snv = smurf_snv[smurf_snv['Chr'] == '22']
smurf_snv['CHROM_POS'] = smurf_snv['Chr'].astype('str').str.cat(smurf_snv['START_POS_REF'].astype('str'),sep="_")
smurf_snv.set_index('CHROM_POS', inplace = True)
smurf_snv = smurf_snv.loc[list(smurf_snv.index[~smurf_snv.index.isin(sample_h.index)])]
print(smurf_snv.shape)

smurf_indels = pd.read_csv('../data/SMURF/NCC_CRC-'+plasmasample1+'-CW/indel-predicted.txt', sep='\t')
smurf_indels = smurf_indels[smurf_indels['Chr'] == '22']
smurf_indels['CHROM_POS'] = smurf_indels['Chr'].astype('str').str.cat(smurf_indels['START_POS_REF'].astype('str'),sep="_")
smurf_indels.set_index('CHROM_POS', inplace = True)
smurf_indels = smurf_indels.loc[list(smurf_indels.index[~smurf_indels.index.isin(sample_h.index)])]
print(smurf_indels.shape)

smurf_calls = pd.concat([smurf_snv[smurf_snv['Chr'] == '22'], smurf_indels[smurf_indels['Chr'] == '22']])
smurf_calls['CHROM_POS'] = smurf_calls['Chr'].astype('str').str.cat(smurf_calls['START_POS_REF'].astype('str'),sep="_")
smurf_calls.set_index('CHROM_POS', inplace = True)
#smurf_calls

In [ ]:
smurf_snv_bis = pd.read_csv('../data/SMURF/NCC_CRC-'+plasmasample2+'-CW/snv-predicted.txt', sep='\t')
smurf_snv_bis = smurf_snv_bis[smurf_snv_bis['Chr'] == '22']
smurf_snv_bis['CHROM_POS'] = smurf_snv_bis['Chr'].astype('str').str.cat(smurf_snv_bis['START_POS_REF'].astype('str'),sep="_")
smurf_snv_bis.set_index('CHROM_POS', inplace = True)
smurf_snv_bis = smurf_snv_bis.loc[list(smurf_snv_bis.index[~smurf_snv_bis.index.isin(sample_h.index)])]
print(smurf_snv_bis.shape)

smurf_indels_bis = pd.read_csv('../data/SMURF/NCC_CRC-'+plasmasample2+'-CW/indel-predicted.txt', sep='\t')
smurf_indels_bis = smurf_indels_bis[smurf_indels_bis['Chr'] == '22']
smurf_indels_bis['CHROM_POS'] = smurf_indels_bis['Chr'].astype('str').str.cat(smurf_indels_bis['START_POS_REF'].astype('str'),sep="_")
smurf_indels_bis.set_index('CHROM_POS', inplace = True)
smurf_indels_bis = smurf_indels_bis.loc[list(smurf_indels_bis.index[~smurf_indels_bis.index.isin(sample_h.index)])]
print(smurf_indels_bis.shape)

smurf_calls_bis = pd.concat([smurf_snv_bis[smurf_snv_bis['Chr'] == '22'], smurf_indels_bis[smurf_indels_bis['Chr'] == '22']])
smurf_calls_bis['CHROM_POS'] = smurf_calls_bis['Chr'].astype('str').str.cat(smurf_calls_bis['START_POS_REF'].astype('str'),sep="_")
smurf_calls_bis.set_index('CHROM_POS', inplace = True)
#smurf_calls

In [ ]:
smurf_snv_t = pd.read_csv('../data/SMURF/NCC_CRC-'+tumorsample1+'/snv-predicted.txt', sep='\t')
smurf_snv_t = smurf_snv_t[smurf_snv_t['Chr'] == '22']
smurf_snv_t['CHROM_POS'] = smurf_snv_t['Chr'].astype('str').str.cat(smurf_snv_t['START_POS_REF'].astype('str'),sep="_")
smurf_snv_t.set_index('CHROM_POS', inplace = True)
smurf_snv_t = smurf_snv_t.loc[list(smurf_snv_t.index[~smurf_snv_t.index.isin(sample_h.index)])]
print(smurf_snv_t.shape)

smurf_indels_t = pd.read_csv('../data/SMURF/NCC_CRC-'+tumorsample1+'/indel-predicted.txt', sep='\t')
smurf_indels_t = smurf_indels_t[smurf_indels_t['Chr'] == '22']
smurf_indels_t['CHROM_POS'] = smurf_indels_t['Chr'].astype('str').str.cat(smurf_indels_t['START_POS_REF'].astype('str'),sep="_")
smurf_indels_t.set_index('CHROM_POS', inplace = True)
smurf_indels_t = smurf_indels_t.loc[list(smurf_indels_t.index[~smurf_indels_t.index.isin(sample_h.index)])]
print(smurf_indels_t.shape)

smurf_calls_t = pd.concat([smurf_snv_t[smurf_snv_t['Chr'] == '22'], smurf_indels_t[smurf_indels_t['Chr'] == '22']])
smurf_calls_t['CHROM_POS'] = smurf_calls_t['Chr'].astype('str').str.cat(smurf_calls_t['START_POS_REF'].astype('str'),sep="_")
smurf_calls_t.set_index('CHROM_POS', inplace = True)
#smurf_calls

# AUPRC with reference = SNV found in undiluted plasma sample by the same method

In [ ]:
a = pd_method[['type_0','type_1', 'type_2', 'type_3', 'type_4']].fillna('').sum(axis=1)
a.value_counts()

In [ ]:
for mutation in ['snv', 'indel']:
    pd_methods = pd.DataFrame
    count = 0

    for mi, method in enumerate(methods):
        #print(method)

        s0 = sample_0[['REF', 'ALT', 'QUAL', 'type', method]]
        s0.rename(columns = {method:'sample_0'},  inplace = True)
        s1 = sample_1[['QUAL', 'type', method]]
        s1.rename(columns = {method:'sample_1'},  inplace = True)
        s2 = sample_2[['QUAL','type', method]]
        s2.rename(columns = {method:'sample_2'},  inplace = True)
        s3 = sample_3[['QUAL','type', method]]
        s3.rename(columns = {method:'sample_3'},  inplace = True)
        s4 = sample_4[['QUAL','type', method]]
        s4.rename(columns = {method:'sample_4'},  inplace = True)

        pd_method = pd.concat([s0, s1, s2, s3, s4], axis=1)

        pd_method.columns = ['REF', 'ALT', 'QUAL_0', 'type_0', 'sample_0', 'QUAL_1', 'type_1', 'sample_1', 'QUAL_2', 'type_2', 'sample_2',  'QUAL_3', 'type_3', 'sample_3', 'QUAL_4', 'type_4', 'sample_4']
        pd_method[['sample_0', 'sample_1', 'sample_2', 'sample_3', 'sample_4']] = pd_method[['sample_0','sample_1', 'sample_2', 'sample_3', 'sample_4']].fillna(value=False)
        pd_method.drop(pd_method[pd_method['sample_0'] + pd_method['sample_1'] + pd_method['sample_2']  + pd_method['sample_3'] + pd_method['sample_4'] == False].index, axis=0, inplace=True)
        pd_method['caller'] = method
        pd_method['type'] = pd_method[['type_0','type_1', 'type_2', 'type_3', 'type_4']].fillna('').sum(axis=1).apply(lambda x: pd.Series(x[:3]))

        if mutation == 'snv':
            pd_method = pd_method[pd_method['type'] == 'SNV']
        elif mutation == 'indel':
            pd_method = pd_method[(pd_method['type'] == 'INS') | (pd_method['type'] == 'DEL')]
                                
        if count == 0:
            pd_methods = pd_method
            count = 1
        else:
            pd_methods = pd.concat([pd_methods, pd_method], join='inner')
    #print(pd_methods.shape)

    pd_results = pd.DataFrame
    pd_results_PR = pd.DataFrame
    count = 0

    fig, axs = plt.subplots(1,5,figsize=(30, 4))
    fig.suptitle('Precision-Recall curves')
    
    if mutation == 'snv': 
        y_true_smurf = pd.Series(True, index=smurf_snv.index)
    elif mutation == 'indel':
        y_true_smurf = pd.Series(True, index=smurf_indels.index)

    y_true_index = pd.Index(list(pd_methods.index)+list(y_true_smurf.index))
    y_true_index = y_true_index.drop_duplicates()
    y_true = pd.Series(False, index=y_true_index)
    y_true.loc[list(y_true_smurf.index)] = True
    print(y_true.shape[0])


    baselineAUPRC = {}


    for mi, method in enumerate(methods):

        res_df = (100*pd.Series(samples_tf)).round(decimals=2).to_frame(name='tumor burden')
        res_PR_df = (100*pd.Series(samples_tf)).round(decimals=2).to_frame(name='tumor burden')

       # y_true = pd_methods[pd_methods['caller'] == method]['sample_0'].astype(bool)
        baselineAUPRC[method] = len(y_true[y_true == True])/len(y_true)
        y_0 = pd_methods[pd_methods['caller'] == method]['sample_0'].astype(bool).reindex(y_true.index).squeeze()
        y_0 = y_0.fillna(False)
        y_1 = pd_methods[pd_methods['caller'] == method]['sample_1'].astype(bool).reindex(y_true.index).squeeze()
        y_1 = y_1.fillna(False)
        y_2 = pd_methods[pd_methods['caller'] == method]['sample_2'].astype(bool).reindex(y_true.index).squeeze()
        y_2 = y_2.fillna(False)
        y_3 = pd_methods[pd_methods['caller'] == method]['sample_3'].astype(bool).reindex(y_true.index).squeeze()
        y_3 = y_3.fillna(False)
        y_4 = pd_methods[pd_methods['caller'] == method]['sample_4'].astype(bool).reindex(y_true.index).squeeze()
        y_4 = y_4.fillna(False)


        res_df['AUPRC'] = [average_precision_score(y_true, y_0),
                          average_precision_score(y_true, y_1),
                          average_precision_score(y_true, y_2),
                          average_precision_score(y_true, y_3),
                          average_precision_score(y_true, y_4),
                         ]
        res_df['AUPRC - baseline AUPRC'] = [average_precision_score(y_true, y_0) - baselineAUPRC[method],
                      average_precision_score(y_true, y_1) - baselineAUPRC[method],
                      average_precision_score(y_true, y_2) - baselineAUPRC[method],
                      average_precision_score(y_true, y_3) - baselineAUPRC[method],
                      average_precision_score(y_true, y_4) - baselineAUPRC[method],
                     ]
        res_PR_df['precision'] = [precision_score(y_true, y_0),
                          precision_score(y_true, y_1),
                          precision_score(y_true, y_2),
                          precision_score(y_true, y_3),
                          precision_score(y_true, y_4),
                         ]

        res_PR_df['recall'] = [recall_score(y_true, y_0),
                          recall_score(y_true, y_1),
                          recall_score(y_true, y_2),
                          recall_score(y_true, y_3),
                          recall_score(y_true, y_4),
                         ]
        res_PR_df = pd.melt(res_PR_df, id_vars =['tumor burden'], value_vars =['precision', 'recall'],
                        var_name='metric', value_name='value')
        res_df['caller'] = method
        res_PR_df['caller'] = method

        res_df.drop(['sample_0'], inplace=True)
        res_PR_df.drop([0, 5], inplace=True)

        # Plot Precision-Recall curve
        alpha_list = [0.4, 1, .75, .5, .3, .1]
        for i in range(1,5):
            y_i = pd_methods[pd_methods['caller'] == method]['sample_'+str(i)].astype(bool).reindex(y_true.index).squeeze()
            y_i = y_i.fillna(False)
            precision, recall, _ = precision_recall_curve(y_true, y_i)
            axs[mi].plot(recall, precision, 'o-',
                                  label='tf='+str(res_df['tumor burden'].loc['sample_'+str(i)])+ ', AP='+str(round(average_precision_score(y_true, y_i), 2)),
                                  c=color_list[mi], alpha=alpha_list[i])
        axs[mi].set_xlabel('Recall')
        axs[mi].set_ylabel('Precision')
        axs[mi].set_ylim([0.0, 1.05])
        axs[mi].set_xlim([0.0, 1.05])
        axs[mi].set_title(method)
        axs[mi].legend()

        if count == 0:
            pd_results = res_df
            pd_results_PR = res_PR_df
            count = 1
        else:
            pd_results = pd.concat([pd_results, res_df], join='inner')
            pd_results_PR = pd.concat([pd_results_PR, res_PR_df], join='inner')
    plt.show()

    plt.figure()
    sns.catplot(x="tumor burden", y="AUPRC", hue="caller",  
                    capsize=.2, height=4, aspect=1.5,
                    kind="point", order=sorted(pd_results['tumor burden'].unique(), reverse=True), data=pd_results)
    for mi, method in enumerate(methods):
        plt.axhline(y = baselineAUPRC[method], color = 'k', linestyle = '--') 
    #plt.ylim([0, 0.5])
    
    sns.catplot(x="tumor burden", y="AUPRC - baseline AUPRC", hue="caller",  
                capsize=.2, height=6, aspect=.75,
                kind="point", order=sorted(pd_results['tumor burden'].unique(), reverse=True), data=pd_results)
    plt.axhline(y = 0, color = 'k', linestyle = '--', label='baseline AUPRC') 

    plt.figure()
    sns.catplot(x="tumor burden", y="value", col="metric", hue="caller",  
                    capsize=.2, height=6, aspect=.75,
                    kind="point", order=sorted(pd_results['tumor burden'].unique(), reverse=True), data=pd_results_PR)
    #plt.ylim([0, 0.5])

# AUPRC with reference = SNV found in both undiluted plasma samples by the same method

In [ ]:
for mutation in ['snv', 'indel']:
    pd_methods = pd.DataFrame
    count = 0

    for mi, method in enumerate(methods):
        #print(method)

        s0 = sample_0[['REF', 'ALT', 'QUAL', 'type', method]]
        s0.rename(columns = {method:'sample_0'},  inplace = True)
        s1 = sample_1[['QUAL', 'type', method]]
        s1.rename(columns = {method:'sample_1'},  inplace = True)
        s2 = sample_2[['QUAL','type', method]]
        s2.rename(columns = {method:'sample_2'},  inplace = True)
        s3 = sample_3[['QUAL','type', method]]
        s3.rename(columns = {method:'sample_3'},  inplace = True)
        s4 = sample_4[['QUAL','type', method]]
        s4.rename(columns = {method:'sample_4'},  inplace = True)

        pd_method = pd.concat([s0, s1, s2, s3, s4], axis=1)

        pd_method.columns = ['REF', 'ALT', 'QUAL_0', 'type_0', 'sample_0', 'QUAL_1', 'type_1', 'sample_1', 'QUAL_2', 'type_2', 'sample_2',  'QUAL_3', 'type_3', 'sample_3', 'QUAL_4', 'type_4', 'sample_4']
        pd_method[['sample_0', 'sample_1', 'sample_2', 'sample_3', 'sample_4']] = pd_method[['sample_0','sample_1', 'sample_2', 'sample_3', 'sample_4']].fillna(value=False)
        pd_method.drop(pd_method[pd_method['sample_0'] + pd_method['sample_1'] + pd_method['sample_2']  + pd_method['sample_3'] + pd_method['sample_4'] == False].index, axis=0, inplace=True)
        pd_method['caller'] = method
        pd_method['type'] = pd_method[['type_0','type_1', 'type_2', 'type_3', 'type_4']].fillna('').sum(axis=1).apply(lambda x: pd.Series(x[:3]))

        if mutation == 'snv':
            pd_method = pd_method[pd_method['type'] == 'SNV']
        elif mutation == 'indel':
            pd_method = pd_method[(pd_method['type'] == 'INS') | (pd_method['type'] == 'DEL')]
                                
        if count == 0:
            pd_methods = pd_method
            count = 1
        else:
            pd_methods = pd.concat([pd_methods, pd_method], join='inner')
    #print(pd_methods.shape)

    pd_results = pd.DataFrame
    pd_results_PR = pd.DataFrame
    count = 0

    fig, axs = plt.subplots(1,5,figsize=(30, 4))
    fig.suptitle('Precision-Recall curves')
    
    if mutation == 'snv': 
        y_true_smurf = pd.Series(True, index=smurf_snv.index)
        y_true_smurf_bis = pd.Series(True, index=smurf_snv_bis.index)
    elif mutation == 'indel':
        y_true_smurf = pd.Series(True, index=smurf_indels.index)
        y_true_smurf_bis = pd.Series(True, index=smurf_indels_bis.index)

    y_true_index = pd.Index(list(pd_methods.index)+list(y_true_smurf.index))
    y_true_index = y_true_index.drop_duplicates()
    y_true_0 = pd.Series(False, index=y_true_index)
    y_true_0.loc[list(y_true_smurf.index)] = True
    print(y_true_0.shape[0])
    
    y_true_index_bis = pd.Index(list(pd_methods.index)+list(y_true_smurf_bis.index))
    y_true_index_bis = y_true_index_bis.drop_duplicates()
    y_true_1 = pd.Series(False, index=y_true_index_bis)
    y_true_1.loc[list(y_true_smurf_bis.index)] = True
    print(y_true_1.shape[0])
    
    y_true_index = pd.Index(list(y_true_0.index)+list(y_true_1.index))
    y_true_index = y_true_index.drop_duplicates()
    y_true = pd.Series(False, index=y_true_index)
    y_true[(y_true_0 == True) & (y_true_1 == True)] = True
    print(y_true.shape[0])
    print(y_true[y_true == True].shape)


    baselineAUPRC = {}


    for mi, method in enumerate(methods):

        res_df = (100*pd.Series(samples_tf)).round(decimals=2).to_frame(name='tumor burden')
        res_PR_df = (100*pd.Series(samples_tf)).round(decimals=2).to_frame(name='tumor burden')

       # y_true = pd_methods[pd_methods['caller'] == method]['sample_0'].astype(bool)
        baselineAUPRC[method] = len(y_true[y_true == True])/len(y_true)
        y_0 = pd_methods[pd_methods['caller'] == method]['sample_0'].astype(bool).reindex(y_true.index).squeeze()
        y_0 = y_0.fillna(False)
        y_1 = pd_methods[pd_methods['caller'] == method]['sample_1'].astype(bool).reindex(y_true.index).squeeze()
        y_1 = y_1.fillna(False)
        y_2 = pd_methods[pd_methods['caller'] == method]['sample_2'].astype(bool).reindex(y_true.index).squeeze()
        y_2 = y_2.fillna(False)
        y_3 = pd_methods[pd_methods['caller'] == method]['sample_3'].astype(bool).reindex(y_true.index).squeeze()
        y_3 = y_3.fillna(False)
        y_4 = pd_methods[pd_methods['caller'] == method]['sample_4'].astype(bool).reindex(y_true.index).squeeze()
        y_4 = y_4.fillna(False)


        res_df['AUPRC'] = [average_precision_score(y_true, y_0),
                          average_precision_score(y_true, y_1),
                          average_precision_score(y_true, y_2),
                          average_precision_score(y_true, y_3),
                          average_precision_score(y_true, y_4),
                         ]
        res_df['AUPRC - baseline AUPRC'] = [average_precision_score(y_true, y_0) - baselineAUPRC[method],
                      average_precision_score(y_true, y_1) - baselineAUPRC[method],
                      average_precision_score(y_true, y_2) - baselineAUPRC[method],
                      average_precision_score(y_true, y_3) - baselineAUPRC[method],
                      average_precision_score(y_true, y_4) - baselineAUPRC[method],
                     ]
        res_PR_df['precision'] = [precision_score(y_true, y_0),
                          precision_score(y_true, y_1),
                          precision_score(y_true, y_2),
                          precision_score(y_true, y_3),
                          precision_score(y_true, y_4),
                         ]

        res_PR_df['recall'] = [recall_score(y_true, y_0),
                          recall_score(y_true, y_1),
                          recall_score(y_true, y_2),
                          recall_score(y_true, y_3),
                          recall_score(y_true, y_4),
                         ]
        res_PR_df = pd.melt(res_PR_df, id_vars =['tumor burden'], value_vars =['precision', 'recall'],
                        var_name='metric', value_name='value')
        res_df['caller'] = method
        res_PR_df['caller'] = method

        res_df.drop(['sample_0'], inplace=True)
        res_PR_df.drop([0, 5], inplace=True)

        # Plot Precision-Recall curve
        alpha_list = [0.4, 1, .75, .5, .3, .1]
        for i in range(1,5):
            y_i = pd_methods[pd_methods['caller'] == method]['sample_'+str(i)].astype(bool).reindex(y_true.index).squeeze()
            y_i = y_i.fillna(False)
            precision, recall, _ = precision_recall_curve(y_true, y_i)
            axs[mi].plot(recall, precision, 'o-',
                                  label='tf='+str(res_df['tumor burden'].loc['sample_'+str(i)])+ ', AP='+str(round(average_precision_score(y_true, y_i), 2)),
                                  c=color_list[mi], alpha=alpha_list[i])
        axs[mi].set_xlabel('Recall')
        axs[mi].set_ylabel('Precision')
        axs[mi].set_ylim([0.0, 1.05])
        axs[mi].set_xlim([0.0, 1.05])
        axs[mi].set_title(method)
        axs[mi].legend()

        if count == 0:
            pd_results = res_df
            pd_results_PR = res_PR_df
            count = 1
        else:
            pd_results = pd.concat([pd_results, res_df], join='inner')
            pd_results_PR = pd.concat([pd_results_PR, res_PR_df], join='inner')
    plt.show()

    plt.figure()
    sns.catplot(x="tumor burden", y="AUPRC", hue="caller",  
                    capsize=.2, height=4, aspect=1.5,
                    kind="point", order=sorted(pd_results['tumor burden'].unique(), reverse=True), data=pd_results)
    for mi, method in enumerate(methods):
        plt.axhline(y = baselineAUPRC[method], color = 'k', linestyle = '--') 
    #plt.ylim([0, 0.5])
    
    sns.catplot(x="tumor burden", y="AUPRC - baseline AUPRC", hue="caller",  
                capsize=.2, height=6, aspect=.75,
                kind="point", order=sorted(pd_results['tumor burden'].unique(), reverse=True), data=pd_results)
    plt.axhline(y = 0, color = 'k', linestyle = '--', label='baseline AUPRC') 

    plt.figure()
    sns.catplot(x="tumor burden", y="value", col="metric", hue="caller",  
                    capsize=.2, height=6, aspect=.75,
                    kind="point", order=sorted(pd_results['tumor burden'].unique(), reverse=True), data=pd_results_PR)
    #plt.ylim([0, 0.5])

## AUPRC with reference = SNV found in the tumor by the same method

In [ ]:
for mutation in ['snv', 'indel']:
    pd_methods = pd.DataFrame
    count = 0

    for mi, method in enumerate(methods):
        #print(method)

        s0 = sample_0[['REF', 'ALT', 'QUAL', 'type', method]]
        s0.rename(columns = {method:'sample_0'},  inplace = True)
        s1 = sample_1[['QUAL', 'type', method]]
        s1.rename(columns = {method:'sample_1'},  inplace = True)
        s2 = sample_2[['QUAL','type', method]]
        s2.rename(columns = {method:'sample_2'},  inplace = True)
        s3 = sample_3[['QUAL','type', method]]
        s3.rename(columns = {method:'sample_3'},  inplace = True)
        s4 = sample_4[['QUAL','type', method]]
        s4.rename(columns = {method:'sample_4'},  inplace = True)

        pd_method = pd.concat([s0, s1, s2, s3, s4], axis=1)

        pd_method.columns = ['REF', 'ALT', 'QUAL_0', 'type_0', 'sample_0', 'QUAL_1', 'type_1', 'sample_1', 'QUAL_2', 'type_2', 'sample_2',  'QUAL_3', 'type_3', 'sample_3', 'QUAL_4', 'type_4', 'sample_4']
        pd_method[['sample_0', 'sample_1', 'sample_2', 'sample_3', 'sample_4']] = pd_method[['sample_0','sample_1', 'sample_2', 'sample_3', 'sample_4']].fillna(value=False)
        pd_method.drop(pd_method[pd_method['sample_0'] + pd_method['sample_1'] + pd_method['sample_2']  + pd_method['sample_3'] + pd_method['sample_4'] == False].index, axis=0, inplace=True)
        pd_method['caller'] = method
        pd_method['type'] = pd_method[['type_0','type_1', 'type_2', 'type_3', 'type_4']].fillna('').sum(axis=1).apply(lambda x: pd.Series(x[:3]))

        if mutation == 'snv':
            pd_method = pd_method[pd_method['type'] == 'SNV']
        elif mutation == 'indel':
            pd_method = pd_method[(pd_method['type'] == 'INS') | (pd_method['type'] == 'DEL')]
                                
        if count == 0:
            pd_methods = pd_method
            count = 1
        else:
            pd_methods = pd.concat([pd_methods, pd_method], join='inner')
    #print(pd_methods.shape)

    pd_results = pd.DataFrame
    pd_results_PR = pd.DataFrame
    count = 0

    fig, axs = plt.subplots(1,5,figsize=(30, 4))
    fig.suptitle('Precision-Recall curves')
    
    if mutation == 'snv': 
        y_true_smurf = pd.Series(True, index=smurf_snv_t.index)
    elif mutation == 'indel':
        y_true_smurf = pd.Series(True, index=smurf_indels_t.index)

    y_true_index = pd.Index(list(pd_methods.index)+list(y_true_smurf.index))
    y_true_index = y_true_index.drop_duplicates()
    y_true = pd.Series(False, index=y_true_index)
    y_true.loc[list(y_true_smurf.index)] = True
    print(y_true.shape[0])


    baselineAUPRC = {}


    for mi, method in enumerate(methods):

        res_df = (100*pd.Series(samples_tf)).round(decimals=2).to_frame(name='tumor burden')
        res_PR_df = (100*pd.Series(samples_tf)).round(decimals=2).to_frame(name='tumor burden')

       # y_true = pd_methods[pd_methods['caller'] == method]['sample_0'].astype(bool)
        baselineAUPRC[method] = len(y_true[y_true == True])/len(y_true)
        y_0 = pd_methods[pd_methods['caller'] == method]['sample_0'].astype(bool).reindex(y_true.index).squeeze()
        y_0 = y_0.fillna(False)
        y_1 = pd_methods[pd_methods['caller'] == method]['sample_1'].astype(bool).reindex(y_true.index).squeeze()
        y_1 = y_1.fillna(False)
        y_2 = pd_methods[pd_methods['caller'] == method]['sample_2'].astype(bool).reindex(y_true.index).squeeze()
        y_2 = y_2.fillna(False)
        y_3 = pd_methods[pd_methods['caller'] == method]['sample_3'].astype(bool).reindex(y_true.index).squeeze()
        y_3 = y_3.fillna(False)
        y_4 = pd_methods[pd_methods['caller'] == method]['sample_4'].astype(bool).reindex(y_true.index).squeeze()
        y_4 = y_4.fillna(False)


        res_df['AUPRC'] = [average_precision_score(y_true, y_0),
                          average_precision_score(y_true, y_1),
                          average_precision_score(y_true, y_2),
                          average_precision_score(y_true, y_3),
                          average_precision_score(y_true, y_4),
                         ]
        res_df['AUPRC - baseline AUPRC'] = [average_precision_score(y_true, y_0) - baselineAUPRC[method],
                      average_precision_score(y_true, y_1) - baselineAUPRC[method],
                      average_precision_score(y_true, y_2) - baselineAUPRC[method],
                      average_precision_score(y_true, y_3) - baselineAUPRC[method],
                      average_precision_score(y_true, y_4) - baselineAUPRC[method],
                     ]
        res_PR_df['precision'] = [precision_score(y_true, y_0),
                          precision_score(y_true, y_1),
                          precision_score(y_true, y_2),
                          precision_score(y_true, y_3),
                          precision_score(y_true, y_4),
                         ]

        res_PR_df['recall'] = [recall_score(y_true, y_0),
                          recall_score(y_true, y_1),
                          recall_score(y_true, y_2),
                          recall_score(y_true, y_3),
                          recall_score(y_true, y_4),
                         ]
        res_PR_df = pd.melt(res_PR_df, id_vars =['tumor burden'], value_vars =['precision', 'recall'],
                        var_name='metric', value_name='value')
        res_df['caller'] = method
        res_PR_df['caller'] = method

        res_df.drop(['sample_0'], inplace=True)
        res_PR_df.drop([0, 5], inplace=True)

        # Plot Precision-Recall curve
        alpha_list = [0.4, 1, .75, .5, .3, .1]
        for i in range(1,5):
            y_i = pd_methods[pd_methods['caller'] == method]['sample_'+str(i)].astype(bool).reindex(y_true.index).squeeze()
            y_i = y_i.fillna(False)
            precision, recall, _ = precision_recall_curve(y_true, y_i)
            axs[mi].plot(recall, precision, 'o-',
                                  label='tf='+str(res_df['tumor burden'].loc['sample_'+str(i)])+ ', AP='+str(round(average_precision_score(y_true, y_i), 2)),
                                  c=color_list[mi], alpha=alpha_list[i])
        axs[mi].set_xlabel('Recall')
        axs[mi].set_ylabel('Precision')
        axs[mi].set_ylim([0.0, 1.05])
        axs[mi].set_xlim([0.0, 1.05])
        axs[mi].set_title(method)
        axs[mi].legend()

        if count == 0:
            pd_results = res_df
            pd_results_PR = res_PR_df
            count = 1
        else:
            pd_results = pd.concat([pd_results, res_df], join='inner')
            pd_results_PR = pd.concat([pd_results_PR, res_PR_df], join='inner')
    plt.show()

    plt.figure()
    sns.catplot(x="tumor burden", y="AUPRC", hue="caller",  
                    capsize=.2, height=4, aspect=1.5,
                    kind="point", order=sorted(pd_results['tumor burden'].unique(), reverse=True), data=pd_results)
    for mi, method in enumerate(methods):
        plt.axhline(y = baselineAUPRC[method], color = 'k', linestyle = '--') 
    #plt.ylim([0, 0.5])
    
    sns.catplot(x="tumor burden", y="AUPRC - baseline AUPRC", hue="caller",  
                capsize=.2, height=6, aspect=.75,
                kind="point", order=sorted(pd_results['tumor burden'].unique(), reverse=True), data=pd_results)
    plt.axhline(y = 0, color = 'k', linestyle = '--', label='baseline AUPRC') 

    plt.figure()
    sns.catplot(x="tumor burden", y="value", col="metric", hue="caller",  
                    capsize=.2, height=6, aspect=.75,
                    kind="point", order=sorted(pd_results['tumor burden'].unique(), reverse=True), data=pd_results_PR)
    #plt.ylim([0, 0.5])

- Histograms of true allele frequencies in each tumor sample. Note how increasing admixture increases the prevalence of low- frequency variants.
- Benchmarking results for germline SNVs
- Benchmarking results for somatic SNVs on exome data.
- averaged over the four replicates

- add fake mutations on healthy mixtures